In [1]:
import casadi

# 例題（式2.35）
## Opti stackを使った書き方

In [2]:
opti = casadi.Opti()

# 変数作成
x1 = opti.variable()
x2 = opti.variable()
x3 = opti.variable()
x4 = opti.variable()
x5 = opti.variable()

In [3]:
opti.minimize(-x1-2*x2) #目的関数(※注意：最大化⇔最小化の変換をしている)

In [4]:
# 制約
opti.subject_to(x1 >= 0)
opti.subject_to(x2 >= 0)
opti.subject_to(x3 >= 0)
opti.subject_to(x4 >= 0)
opti.subject_to(x5 >= 0)
opti.subject_to(x1 + x2 + x3 == 6)
opti.subject_to(x1 + 3*x2 + x4 == 12)
opti.subject_to(2*x1 + x2 + x5 == 10)

In [5]:
p_opts = {"expand":True}
s_opts = {"max_iter": 100}
opti.solver("ipopt", p_opts, s_opts)

In [6]:
sol = opti.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        9
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [7]:
print(sol.value(x1), sol.value(x2))

3.0000000050350324 2.9999999999820606


- 答えあってるっぽい

## Symbolic frameworkを使った書き方
- 式2.35を行列表現になおしてみる
    - 目的関数:$(1~2~0~0~0)~ \boldsymbol{x}$
    - 制約:
$\left(
    \begin{array}{ccccc}
      1 & 1 & 1 & 0 & 0\\
      1 & 3 & 0 & 1 & 0\\
      2 & 1 & 0 & 0 & 1\\
      0 & 0 & 0 & 0 & 0\\
      0 & 0 & 0 & 0 & 0
    \end{array}
  \right)\boldsymbol{x}- \left(
    \begin{array}{c}
      6 \\
      12 \\
      10 \\
      0 \\
      0 
    \end{array}
  \right)=\boldsymbol{0}$

In [8]:
# 行列表現
b = [6,12,10,0,0]
c = [-1,-2,0,0,0] #問題だと最大化だがソルバーは最小化のみなので-1かける
D = [[1,1,1,0,0],[1,3,0,1,0],[2,1,0,0,1],[0,0,0,0,0],[0,0,0,0,0]]

In [9]:
x = casadi.SX.sym("x", 5) #代数生成（縦ベクトル）
b = casadi.SX(b) # symメソッドを使わないと定数になる 
c = casadi.SX(c)
D = casadi.SX(D) 

In [10]:
f = casadi.mtimes(c.T, x) # 目的関数　内積
g = casadi.mtimes(D,x) - b

In [11]:
nlp = {} #変数、目的関数、制約は辞書型で渡す
nlp["x"] = x # 複数変数がある場合はcasadi.vertcat()関数を使ってひとつの構造体にする
nlp["f"] = f 
nlp["g"] = g

In [12]:
F = casadi.nlpsol("F", "ipopt", nlp) # ファンクションとして登録

In [13]:
sol = F(x0=[0,0,0,0,0],ubg=0,lbg=0, lbx=0)
# 引数 x0:変数の初期値、ubg:制約上限＆lbg:制約下限（どっちも0で等式制約）、ubx:変数上限＆lbx:変数下限

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        9
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        5
                     variables with only lower bounds:        5
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

In [14]:
sol["x"][:2]

DM([2.9983, 2.9972])

- うまくいってるっぽい

# 演習問題